# **STEMfit example**
## *Center of mass mode*

This notebook gives a short demonstration of how to detect atoms in a STEM image and determine their positions using center of mass mode. First, we start by importing `STEMfit`:

In [ ]:
using Pkg; Pkg.activate("..");
import STEMfit

Now, we load our example image. The image we use here is of three oxide perovskite layers grown epitaxially on each other. By setting `convert=true`, we let `STEMfit` convert our 16 bit TIFF image into the required format.

In [ ]:
image = STEMfit.load_image("image.tif")

We filter the raw image using a combination of singular value decomposition and Gaussian convolution. `STEMfit` will automatically determine the number of singular vectors to use. For diagnostic purposes, we can set the keyword argument `plot=true`:

In [ ]:
filtered_image = STEMfit.filter_image(image, plot=false)

Then, we use the `find_atoms` function to find atoms in the image using an adaptive thresholding procedure. The function takes the image as an argument as well as `bias` and `window_size` parameters which control the thresholding. Increasing `bias` tends to make the threshold higher (i.e. more separation of atoms), `window_size` determines how big of an area is considered at each point in the image. The function returns a matrix of atom positions, a vector of atom widths, a vector of atom intensities and the thresholded image. The latter can be used to assess the quality of the thresholding and asjust the `bias` and `window_size` parameters. 

In [ ]:
tp = STEMfit.ThresholdingParameters(bias=0.5, window_size=5, minimum_atom_size=0)


In [ ]:
(atom_parameters, thresholded_image) = STEMfit.find_atoms(filtered_image, tp);

# **Processing**

In [ ]:
(unit_cells, neighbors, atom_tree) = STEMfit.find_unit_cells(
                                    atom_parameters, 
                                    uc_allowed_angles=85:95, #Only orthogonal unit cells
                                    uc_allowed_orientation=175:185 #Only horizontal unit cells
                                    );

In [ ]:
results = STEMfit.calculate_lattice_parameters(atom_parameters, unit_cells[2])

In [ ]:
known_lattice_parameters = (0.3905, 0.3905)
reference_range_y = 0.8*size(image, 1):size(image, 1)
reference_range_x = 0:size(image, 2)

STEMfit.add_pixel_size(
    results, 
    reference_range_y, 
    reference_range_x, 
    known_lattice_parameters
    )


In [ ]:
ellipticity_map, angle_map = STEMfit.map_ellipticity(results, markersize=2, size=(1000,400));

In [ ]:
lp_histogram = STEMfit.lattice_parameter_histogram(results);

In [ ]:
(lp_map_1, lp_map_2) = STEMfit.map_lattice_parameter(results, markersize=2);

In [ ]:
STEMfit.save_atomic_parameters("Results/results.csv", results)

STEMfit.save("Results/binarized image.png", thresholded_image)
STEMfit.save("Results/filtered image.png", filtered_image)

STEMfit.savefig(lp_histogram, "Results/lattice parameter histogram.png")

STEMfit.savefig(lp_map_1, "Results/lattice parameter map 1.png")
STEMfit.savefig(lp_map_2, "Results/lattice parameter map 2.png")

STEMfit.savefig(ellipticity_map, "Results/strain map 1.png")
STEMfit.savefig(angle_map, "Results/strain map 2.png");

In [ ]:
blp = Dict(1=>(1,1), 2=>(2,2))
STEMfit.calculate_true_strain(results, blp, [600,])

In [ ]:
results.strain

In [ ]:
results.layer_assignments